Version 1

In [25]:
import cv2
import numpy as np
import os
import tkinter as tk
from tkinter import filedialog, ttk

class LucasKanade_OpticalFlow:
    def __init__(self, video_path=None):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path) if self.video_path else None
        self.output_folder = None
        self.frame_count = 0

    def create_output_folder(self):
        folder_number = 1

        while True:
            current_folder = f'optical_flow_images_{folder_number:03d}'
            if not os.path.exists(current_folder):
                os.makedirs(current_folder)
                self.output_folder = current_folder
                break
            else:
                folder_number += 1

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.001, minDistance=0.1)
        return corners, gray

    def calculate_optical_flow(self, prev_gray, gray, prev_pts):
        new_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, winSize=(25, 25), maxLevel=3,
                                                        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
        flow_image = np.zeros_like(prev_gray)
        for new, old in zip(new_pts, prev_pts):
            a, b = new.ravel()
            c, d = old.ravel()
            velocity_x = a - c
            velocity_y = b - d
            intensity = int(np.sqrt(velocity_x**2 + velocity_y**2) * 255 / 5)
            color = (intensity, 0, 0)
            flow_image = cv2.line(flow_image, (int(c), int(d)), (int(a), int(b)), color, 2)
        return flow_image

    def track_features(self):
        self.get_video_path
        if not self.cap:
            return

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            cv2.imshow('Optical Flow', flow_image)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

        self.cap.release()
        cv2.destroyAllWindows()

    def get_video_path(self):
        file_path = filedialog.askopenfilename(title="Select Video File", filetypes=[("Video files", "*.mp4;*.avi")])
        if file_path:
            self.video_path = file_path
            self.cap = cv2.VideoCapture(self.video_path)

    def get_optical_images(self, progress_var):
        self.get_video_path()
        if not self.cap:
            return

        progress_window = tk.Toplevel()  
        progress_window.title("Optical Flow Conversion Progress")
        screen_width = progress_window.winfo_screenwidth()
        screen_height = progress_window.winfo_screenheight()
        x_coordinate = (screen_width - 500) // 2
        y_coordinate = (screen_height - 500) // 2
        progress_window.geometry(f"500x200+{x_coordinate}+{y_coordinate}")

        progress_label = tk.Label(progress_window, text="Optical Flow Conversion Progress:")
        progress_label.pack(pady=10)

        progress_bar = ttk.Progressbar(progress_window, variable=progress_var, length=200, mode='determinate')
        progress_bar.pack(pady=10)

        if not self.cap:
            return

        self.create_output_folder()

        total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))

        ret, frame = self.cap.read()
        prev_corners, prev_gray = self.detect_corners(frame)

        while ret:
            ret, frame = self.cap.read()
            if not ret:
                break

            new_corners, gray = self.detect_corners(frame)
            flow_image = self.calculate_optical_flow(prev_gray, gray, prev_corners)

            cv2.imwrite(os.path.join(self.output_folder, f'optical_flow_{self.frame_count + 1:04d}.png'), flow_image)

            progress_var.set(int((self.frame_count / total_frames) * 100))  # Update progress

            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            self.frame_count += 1
            prev_gray = gray
            prev_corners = new_corners.reshape(-1, 1, 2)

            progress_window.update()

        self.frame_count = 0
        self.cap.release()
        cv2.destroyAllWindows()

        # Close the progress bar window
        progress_window.destroy()

    def show_optical_flow_video(self):
        self.get_video_path()
        self.track_features()

    def live_optical_flow_tracking(self):
        self.cap = cv2.VideoCapture(0)  # Use camera feed
        self.track_features()

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Lucas-Kanade Optical Flow")
    # Center the root window on the screen
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x_coordinate = (screen_width - 500) // 2
    y_coordinate = (screen_height - 500) // 2
    root.geometry(f"500x200+{x_coordinate}+{y_coordinate}")
    feature_tracker = LucasKanade_OpticalFlow()

    def get_optical_images():
        progress_var = tk.IntVar()
        progress_var.set(0)
        feature_tracker.get_optical_images(progress_var)

    def show_optical_flow_video():
        feature_tracker.show_optical_flow_video()

    def live_optical_flow_tracking():
        feature_tracker.live_optical_flow_tracking()

    def exit_application():
        root.destroy()

    # Buttons
    btn_get_optical_images = tk.Button(root, text="Get Optical Images", command=get_optical_images)
    btn_get_optical_images.pack(pady=10)

    btn_show_optical_flow = tk.Button(root, text="Show Optical Flow Video", command=show_optical_flow_video)
    btn_show_optical_flow.pack(pady=10)

    btn_live_tracking = tk.Button(root, text="Live Optical Flow Tracking", command=live_optical_flow_tracking)
    btn_live_tracking.pack(pady=10)

    btn_exit = tk.Button(root, text="Exit", command=exit_application)
    btn_exit.pack(pady=10)

    root.mainloop()
